ERP is an acronym that stands for **Enterprise Resource Planning** (ERP). 

It's a business process management software that manages and integrates a company's financials, supply chain, operations, commerce, reporting, manufacturing, and human resource activities. 

One important ERP entity is the **Account Receivable (AR)**: it refers to the money a company's customers owe for goods or services they have received.

 **Account Receivable (AR)** could be:
 * Invoice
 * Credit Note
 * Debit Note
 * Cancellation
 * Miscellaneous

Each AR is made by several part like the **header** – the part with general information about customers/suppliers that define the invoice – the **list of items**, the **list of payments**, details about the **customers**, details about the **shipping**, ...

# Parameters

* N: number of invoices
* M: number of payments
* K: number of customers

In [52]:
N=10000
M=12500
K=150

# AR Header

The Header of an AR document contains some general information like
* Customer ID
* Value
* Due Date
* Posting Date
* Document Number - must be unique per fiscal year
* Fiscal Year
* Document Type

Assumptions:
* we have "Invoice" has only type

In [53]:
from random import randint
from datetime import datetime,timedelta
def headerGenerator(k=5):
  postingDate = datetime(2022,1,1)+timedelta(randint(0,200))
  return {
          "customerId":"Customer_{customerId}".format(customerId=str(randint(0,k)+1).zfill(3)),
          "value":randint(50,10000),
          "documentCurrency":"EUR", 
          "postingDate":postingDate.strftime("%Y-%m-%d"),
          "dueDate":(postingDate+timedelta(randint(0,60))).strftime("%Y-%m-%d"),
          "fiscalYear":postingDate.strftime("%Y"),
          "documentType":"Invoice"
         }


def headerList(k=5,n=1000):
  rawHeaderList = [headerGenerator(k) for k in range(n)]
  rawHeaderList.sort(key=lambda row: row.get("postingDate"))
  for pos,val in enumerate(rawHeaderList):
    val["documentNumber"]="2022-{docNum}".format(docNum=str(pos).zfill(5))
  return rawHeaderList
  
myARList = headerList(K,N)
myARList

[{'customerId': 'Customer_172',
  'value': 2815,
  'documentCurrency': 'EUR',
  'postingDate': '2022-01-01',
  'dueDate': '2022-02-11',
  'fiscalYear': '2022',
  'documentType': 'Invoice',
  'documentNumber': '2022-00000'},
 {'customerId': 'Customer_178',
  'value': 5653,
  'documentCurrency': 'EUR',
  'postingDate': '2022-01-01',
  'dueDate': '2022-02-08',
  'fiscalYear': '2022',
  'documentType': 'Invoice',
  'documentNumber': '2022-00001'},
 {'customerId': 'Customer_352',
  'value': 8440,
  'documentCurrency': 'EUR',
  'postingDate': '2022-01-01',
  'dueDate': '2022-01-26',
  'fiscalYear': '2022',
  'documentType': 'Invoice',
  'documentNumber': '2022-00002'},
 {'customerId': 'Customer_690',
  'value': 8872,
  'documentCurrency': 'EUR',
  'postingDate': '2022-01-01',
  'dueDate': '2022-01-13',
  'fiscalYear': '2022',
  'documentType': 'Invoice',
  'documentNumber': '2022-00003'},
 {'customerId': 'Customer_715',
  'value': 2297,
  'documentCurrency': 'EUR',
  'postingDate': '2022-01-

# AR Payments

List of lines that represent a payment made by a customer on a given AR.
* Document Number
* Payment Date
* Value Paid

In [54]:
def paymentGenerator(InvoiceList):
  documentNumber = "2022-{docNum}".format(docNum=str(randint(0,len(InvoiceList)-1)).zfill(5))
  invoice = [k for k in InvoiceList if k.get("documentNumber")==documentNumber][0]
  postingDate = datetime.strptime(invoice.get("postingDate"),"%Y-%m-%d")
  return { 
          "documentNumber":documentNumber,
          "paymentDate":(postingDate+timedelta(randint(15,90))).strftime("%Y-%m-%d"),
          "valuePaid":randint(1,invoice.get("value"))
          ,"documentCurrency":invoice.get("documentCurrency")
         }


def paymentList(InvoiceList,m=250):
  return [paymentGenerator(InvoiceList) for k in range(m)]
   
myPaymentList = paymentList(myARList,M)  
myPaymentList

[{'documentNumber': '2022-05957',
  'paymentDate': '2022-07-03',
  'valuePaid': 5314,
  'documentCurrency': 'EUR'},
 {'documentNumber': '2022-00727',
  'paymentDate': '2022-03-01',
  'valuePaid': 6614,
  'documentCurrency': 'EUR'},
 {'documentNumber': '2022-02678',
  'paymentDate': '2022-03-22',
  'valuePaid': 1090,
  'documentCurrency': 'EUR'},
 {'documentNumber': '2022-05546',
  'paymentDate': '2022-07-16',
  'valuePaid': 556,
  'documentCurrency': 'EUR'},
 {'documentNumber': '2022-01970',
  'paymentDate': '2022-03-27',
  'valuePaid': 683,
  'documentCurrency': 'EUR'},
 {'documentNumber': '2022-00795',
  'paymentDate': '2022-02-15',
  'valuePaid': 2805,
  'documentCurrency': 'EUR'},
 {'documentNumber': '2022-00681',
  'paymentDate': '2022-02-07',
  'valuePaid': 3120,
  'documentCurrency': 'EUR'},
 {'documentNumber': '2022-05764',
  'paymentDate': '2022-06-25',
  'valuePaid': 195,
  'documentCurrency': 'EUR'},
 {'documentNumber': '2022-03110',
  'paymentDate': '2022-06-01',
  'valuePa

# Part 00
* Define the type of each table (Log or Registry): which are the keys of these tables?

Both Header and Payments are log, because I cannot update or delete no one of them.
Keys:
* Header: documentNumber and fiscalYear
* Payments: documentNumber and paymenteDate (under the assumpions: i) I can receive multiple payments for a given AR, ii) I cannot receive more than one payment for a given invoice per day

# Part 01
* Create the two RDDs checking everything is ok!
* Create a unique RDD with pieces of information both from header and payments

In [4]:
#import pyspark
#sc = pyspark.SparkContext("local[*]")

22/11/27 11:25:32 WARN Utils: Your hostname, federicobruzzone resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface enp5s0)
22/11/27 11:25:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/27 11:25:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [68]:
headerRDD = sc.parallelize(myARList)
paymentsRDD = sc.parallelize(myPaymentList)
print(headerRDD.first(), "\n")
print(paymentsRDD.first(), "\n")
headerRDD.count()==N,paymentsRDD.count()==M

{'customerId': 'Customer_172', 'value': 2815, 'documentCurrency': 'EUR', 'postingDate': '2022-01-01', 'dueDate': '2022-02-11', 'fiscalYear': '2022', 'documentType': 'Invoice', 'documentNumber': '2022-00000'} 

{'documentNumber': '2022-05957', 'paymentDate': '2022-07-03', 'valuePaid': 5314, 'documentCurrency': 'EUR'} 



(True, True)

In [69]:
semiJoinHeaderRDD = headerRDD.map(lambda row:((row.get("documentNumber"), row.get("fiscalYear")),row))
semiJoinPaymentsRDD = paymentsRDD.map(lambda row:((row.get("documentNumber"), datetime.strptime(row.get("paymentDate"),"%Y-%m-%d").strftime("%Y")),row))
print(semiJoinHeaderRDD.first(), "\n")
print(semiJoinPaymentsRDD.first(), "\n")
joinRDD = semiJoinPaymentsRDD.join(semiJoinHeaderRDD)
print(joinRDD.first())

(('2022-00000', '2022'), {'customerId': 'Customer_172', 'value': 2815, 'documentCurrency': 'EUR', 'postingDate': '2022-01-01', 'dueDate': '2022-02-11', 'fiscalYear': '2022', 'documentType': 'Invoice', 'documentNumber': '2022-00000'}) 

(('2022-05957', '2022'), {'documentNumber': '2022-05957', 'paymentDate': '2022-07-03', 'valuePaid': 5314, 'documentCurrency': 'EUR'}) 



(('2022-03011', '2022'), ({'documentNumber': '2022-03011', 'paymentDate': '2022-04-14', 'valuePaid': 492, 'documentCurrency': 'EUR'}, {'customerId': 'Customer_4732', 'value': 824, 'documentCurrency': 'EUR', 'postingDate': '2022-03-04', 'dueDate': '2022-03-20', 'fiscalYear': '2022', 'documentType': 'Invoice', 'documentNumber': '2022-03011'}))


In [28]:
# Test
#semiJoinPaymentsRDD.join(semiJoinHeaderRDD.union(sc.parallelize(["pippo"]))).first()
#semiJoinHeaderRDD.union(sc.parallelize(["pippo"])).filter(lambda row: type(row)==tuple and type(row[1])==dict).join(semiJoinPaymentsRDD).count()

12500

In [70]:
#joinRDD.filter(lambda value: joinRDD.collect()[1].get("paymentDate") < joinRDD.collect()[1][1].get("postingDate"))

x = joinRDD.first()
getPaymentDate = x[1][0].get("paymentDate")
datetimeGetPaymentDate = datetime.strptime(x[1][0].get("paymentDate"), '%Y-%m-%d')
getDueDate = x[1][1].get("dueDate")
datetimeGetDueDate = datetime.strptime(x[1][1].get("dueDate"), '%Y-%m-%d')
print(getPaymentDate, "<", getDueDate, "?", getPaymentDate < getDueDate)
print(getPaymentDate, ">", getDueDate, "?", getPaymentDate > getDueDate)
print(getPaymentDate, "==", datetimeGetPaymentDate, "?", getPaymentDate==datetimeGetPaymentDate)
joinRDD.first()

2022-04-14 < 2022-03-20 ? False
2022-04-14 > 2022-03-20 ? True
2022-04-14 == 2022-04-14 00:00:00 ? False


(('2022-03011', '2022'),
 ({'documentNumber': '2022-03011',
   'paymentDate': '2022-04-14',
   'valuePaid': 492,
   'documentCurrency': 'EUR'},
  {'customerId': 'Customer_4732',
   'value': 824,
   'documentCurrency': 'EUR',
   'postingDate': '2022-03-04',
   'dueDate': '2022-03-20',
   'fiscalYear': '2022',
   'documentType': 'Invoice',
   'documentNumber': '2022-03011'}))

# Part 02
* How many invoices are open (i.e., not completely paid)?
* How many invoices are closed (i.e., completely paid)?
* How many invoices are overdued (i.e., not completely paid and with a due date in the past)?
* How many invoices have been paid not in time (i.e., completely paid and with the last payment after the due date)?
* Add to the RDD the information of "closingDate" as the date of the payment that close that invoice.
* Add to the RDD the boolean of "inTime": True if the closingDate < dueDate else False

In [129]:
def formatRow(row):
    print(row)
    newRow = {"key" : row[0]}
    newRow["header"] = row[1][1]
    newRow["payments"] = [row[1][0]]
    return newRow
fJoinRDD = joinRDD.map(lambda row: formatRow(row))

def removeKey(row, key): 
    del row[key]
    return row
def mergePayments(l,r):
    l["payments"] += r["payments"]
    return l
mJoinRDD = fJoinRDD.map(lambda row: (row.get("key"), removeKey(row, "key"))) \
                   .reduceByKey(lambda l,r : mergePayments(l,r))

In [143]:
# How many invoices are open (i.e., not completely paid)?
def isOpen(row):
    value = row[1].get("header").get("value")
    valuePaid = sum([payment.get("valuePaid") for payment in row[1].get("payments")])
    return valuePaid < value
mJoinRDD.filter(lambda row : isOpen(row)).count()

4795

# Part 03 - Debit Note
* How many invoices have been paid for more then their value?
* Add to the Header RDD for each of them a Debit Note with the value to be charged back and the date of today

# Part 04 - Paymenets Frequency
* Add to the Payment Rdd the computed "expectedPaymentDate". It is based on the two previous payments, and is the last payment date + the difference between it and the payment right before, customer by customer.
So, in the example below, for the first two payment is not possible to compute, while for the third, the expected payment is the 2022/10/15 (date of the last payment) plus 3 (the difference between it and the payment of 2022/10/12) 
| customerId  | paymentDate | expectedPaymentDate | documentNumber | ... |
|-------------|-------------|---------------------|----------------|-----|
| Customer001 | 2022/10/12  | N/A                 | 2022_01001     | ... |
| Customer001 | 2022/10/15  | N/A                 | 2022_01004     | ... |
| Customer001 | 2022/10/16  | 2022/10/18 (15+3)   | 2022_00904     | ... |
| Customer001 | 2022/10/20  | 2022/10/17 (16+1)   | 2022_01004     | ... |
| Customer001 | 2022/10/30  | 2022/11/24 (20+4)   | 2022_01101     | ... |
| Customer001 | ...         | ...                 | ...            | ... |
* Show for each customer, the average error of such method

# Part 05 - Cosine Similarity
* How many customers has the company?
* Draw the histogram - without using .hist() - as the number of customer with 1 invoice, the number of customers with 2 invoices, ...
* Define two customers similarity based on the cosine similarity computed on the average payment time per day
    * a day with no invoice posted count as zero
    * for other days, compute the average payment timing using the due date as zero (10 days in advance means -10, 3 days after means +3)